In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
def find_similar_movies(items_emb_final, movie_index, top_k=10):
    # 获取目标电影的嵌入
    target_emb = items_emb_final[movie_index]  # shape: (embedding_dim,)
    
    # 计算余弦相似度
    norm_items_emb = items_emb_final / np.linalg.norm(items_emb_final, axis=1, keepdims=True)  # 单位化每个电影的嵌入
    norm_target_emb = target_emb / np.linalg.norm(target_emb)  # 单位化目标电影的嵌入
    similarities = np.dot(norm_items_emb, norm_target_emb)  # shape: (num_movies,)
    
    # 排除目标电影本身
    similarities[movie_index] = -1.0  # 设置自身的相似度为 -1，避免被选中
    
    # 获取 Top-K 相似电影
    topk_indices = np.argsort(similarities)[-top_k:][::-1]  # 从大到小排序并取前K个索引
    
    return topk_indices.tolist()

In [8]:
# 加载时使用以下代码
# 加载 embeddings
dataset = 'ml-25m-reduced'
users_embeddings_file = './save/'+dataset+'/users_embeddings.npy'
items_embeddings_file = './save/'+dataset+'/items_embeddings.npy'
maps_file = './save/'+dataset+'/dataset_maps.pkl'
users_embeddings = np.load(users_embeddings_file)
items_embeddings = np.load(items_embeddings_file)

# 加载 maps
with open(maps_file, 'rb') as f:
    loaded_maps = pickle.load(f)

movie_map = loaded_maps['movie_map']
user_map = loaded_maps['user_map']

print("Embeddings and maps loaded successfully.")

# 为指定电影推荐相似电影
movies_df = pd.read_csv("../data/"+dataset+"/movies.csv")
movie_index  = 0
similar_indices = find_similar_movies(items_embeddings, movie_index, top_k=10)

movie_mapping_inverse = {v: k for k, v in movie_map.items()}
target_movie_name = movies_df[movies_df['movieId'] == movie_mapping_inverse[movie_index]]['title'].values[0]
target_movie_type = movies_df[movies_df['movieId'] == movie_mapping_inverse[movie_index]]['genres'].values[0]
similar_ids = [movie_mapping_inverse[indice] for indice in similar_indices]
filtered_df = movies_df[movies_df['movieId'].isin(similar_ids)]
similar_movie_names = filtered_df['title'].tolist()
similar_movie_types = filtered_df['genres'].tolist()

print(f"Target movie: {target_movie_name} ({target_movie_type})")
print(f"Similar movies:")
for name, genre in zip(similar_movie_names, similar_movie_types):
    print(f"{name} ({genre})")



Embeddings and maps loaded successfully.
Target movie: Toy Story (1995) (Adventure|Animation|Children|Comedy|Fantasy)
Similar movies:
Lion King, The (1994) (Adventure|Animation|Children|Drama|Musical|IMAX)
Aladdin (1992) (Adventure|Animation|Children|Comedy|Musical)
Toy Story 2 (1999) (Adventure|Animation|Children|Comedy|Fantasy)
Monsters, Inc. (2001) (Adventure|Animation|Children|Comedy|Fantasy)
Finding Nemo (2003) (Adventure|Animation|Children|Comedy)
Incredibles, The (2004) (Action|Adventure|Animation|Children|Comedy)
Ratatouille (2007) (Animation|Children|Drama)
WALL·E (2008) (Adventure|Animation|Children|Romance|Sci-Fi)
Up (2009) (Adventure|Animation|Children|Drama)
Toy Story 3 (2010) (Adventure|Animation|Children|Comedy|Fantasy|IMAX)
